In [9]:
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.models import Sequential
from keras import backend as K
 
img_width, img_height = 128, 128

input_shape = (img_width, img_height, 3) ## we checked that the RGB channel comes last in the data format of these images

epochs = 10
batch_size = 16

In [21]:
model = Sequential()
model.add(Conv2D(32, (2, 2), input_shape = input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
 
model.add(Conv2D(32, (2, 2)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
 
model.add(Conv2D(64, (2, 2)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
 
model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(4)) ## 4 output classes
model.add(Activation('sigmoid'))

model.compile(loss = 'binary_crossentropy',
              optimizer = 'rmsprop',
              metrics = ['accuracy']) ## try accuracy for now, can change later

In [27]:
# previously we used the datagen.flow_from_directory function for ease of loading data for the model even though
# it is technically a data augmentation method; this may be outdated so let's try image_dataset_from_directory
 
train_data = tf.keras.utils.image_dataset_from_directory(
    r'C:\Users\lucia\OneDrive - University of Calgary\University\Year 3\Semester 2\Bmen 415\Coursework\Course Project\GitHub\BMEN-415\Data\_split datasets\image train_test_val split\train',
    image_size = (img_width, img_height),
    batch_size = batch_size,
    labels='inferred',
    label_mode = 'categorical',
    class_names = ['Mild_Demented', 'Moderate_Demented', 'Non_Demented', 'Very_Mild_Demented'],
    seed = 112)
 
validation_data = tf.keras.utils.image_dataset_from_directory(
    r'C:\Users\lucia\OneDrive - University of Calgary\University\Year 3\Semester 2\Bmen 415\Coursework\Course Project\GitHub\BMEN-415\Data\_split datasets\image train_test_val split\val',
    image_size = (img_width, img_height),
    batch_size = batch_size,
    labels='inferred',
    label_mode = 'categorical',
    class_names = ['Mild_Demented', 'Moderate_Demented', 'Non_Demented', 'Very_Mild_Demented'],
    seed = 112)
 
model.fit(
    train_data,
    steps_per_epoch = 4487 // (batch_size*epochs),
    epochs = epochs,
    validation_data = validation_data,
    validation_steps = 642 // (batch_size*epochs))

    ## NOTE: took out the class weights *for now* since the model accuracy seems to be higher without it
    ## class_weight = {0: 4487/627, 1: 4487/44, 2: 4487/2240, 3: 4487/1568}

Found 4487 files belonging to 4 classes.
Found 642 files belonging to 4 classes.
Epoch 1/10
28/28 [==============================] - 4s 121ms/step - loss: 0.4731 - accuracy: 0.5357 - val_loss: 0.4333 - val_accuracy: 0.5469
Epoch 2/10
28/28 [==============================] - 3s 121ms/step - loss: 0.4623 - accuracy: 0.4799 - val_loss: 0.4294 - val_accuracy: 0.5312
Epoch 3/10
28/28 [==============================] - 3s 124ms/step - loss: 0.4610 - accuracy: 0.5134 - val_loss: 0.4399 - val_accuracy: 0.4688
Epoch 4/10
28/28 [==============================] - 3s 123ms/step - loss: 0.4430 - accuracy: 0.5424 - val_loss: 0.4510 - val_accuracy: 0.5156
Epoch 5/10
28/28 [==============================] - 3s 122ms/step - loss: 0.4694 - accuracy: 0.5312 - val_loss: 0.4412 - val_accuracy: 0.5000
Epoch 6/10
28/28 [==============================] - 3s 124ms/step - loss: 0.4404 - accuracy: 0.5603 - val_loss: 0.4437 - val_accuracy: 0.5000
Epoch 7/10
28/28 [==============================] - 3s 124ms/step -

In [28]:
model.save('CNN_model.h5')

In [32]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.applications.vgg16 import decode_predictions
from tensorflow.keras.applications.vgg16 import VGG16
import numpy as np
 
from keras.models import load_model
 
model = load_model('CNN_model.h5')
 
## let's test one of the mild demented images reserved in the training dataset
image = load_img(r'C:\Users\lucia\OneDrive - University of Calgary\University\Year 3\Semester 2\Bmen 415\Coursework\Course Project\_FINAL DATASETS\alzheimer mri preprocessed dataset\train_test_val split\test\Mild_Demented\mild_55.jpg', target_size = (128, 128))
img = np.array(image) / 255
img = img.reshape(1, 128, 128, 3)

prediction = model.predict(img)
print(prediction)
## these are probabilities, i.e. the predicted class is 2 = moderate demented?

1/1 [==============================] - 0s 81ms/step
[[0.3756     0.4137411  0.39648595 0.3891413 ]]
